# Merging table to create Dataset for feature engineering

In [99]:
import pandas as pd
import numpy as np
import datetime as dt

pd.set_option('display.max_columns', None)

In [100]:
# Data import
games = pd.read_csv("../raw_data/games_w_venue.csv", index_col="Unnamed: 0")
players = pd.read_csv("../raw_data/players.csv", index_col="Unnamed: 0")
stadiums = pd.read_csv("../raw_data/stadiums.csv")
teams = pd.read_csv("../raw_data/teams.csv")
data = pd.read_csv("../raw_data/all_ab_raw_data_w_target.csv")

In [101]:
data.shape

(142307, 25)

In [102]:
# Merging data and games
games = games.rename(columns={"id": "game_id"})
data = data.merge(games, how="left", on='game_id')
data.shape

(142307, 43)

In [103]:
pitch_pref_data = []
for idh in players.id:
    
    h = data[data["hitter_id"] == idh]

    zone_1 = h[h["pitch_location_zone"] == 1]["y_target"].mean()
    zone_2 = h[h["pitch_location_zone"] == 2]["y_target"].mean()
    zone_3 = h[h["pitch_location_zone"] == 3]["y_target"].mean()
    zone_4 = h[h["pitch_location_zone"] == 4]["y_target"].mean()
    zone_5 = h[h["pitch_location_zone"] == 5]["y_target"].mean()
    zone_6 = h[h["pitch_location_zone"] == 6]["y_target"].mean()
    zone_7 = h[h["pitch_location_zone"] == 7]["y_target"].mean()
    zone_8 = h[h["pitch_location_zone"] == 8]["y_target"].mean()
    zone_9 = h[h["pitch_location_zone"] == 9]["y_target"].mean()
    zone_10 = h[h["pitch_location_zone"] == 10]["y_target"].mean()
    zone_11 = h[h["pitch_location_zone"] == 11]["y_target"].mean()
    zone_12 = h[h["pitch_location_zone"] == 12]["y_target"].mean()
    zone_13 = h[h["pitch_location_zone"] == 13]["y_target"].mean()
    zone_14 = h[h["pitch_location_zone"] == 14]["y_target"].mean()
    

    zone_eff_h = {'id': idh,
                'zone_1': zone_1,
                'zone_2': zone_2,
                'zone_3': zone_3,
                'zone_4': zone_4,
                'zone_5': zone_5,
                'zone_6': zone_6,
                'zone_7': zone_7,
                'zone_8': zone_8,
                'zone_9': zone_9,
                'zone_10': zone_10,
                'zone_11': zone_11,
                'zone_12': zone_12,
                'zone_13': zone_13,
                'zone_14': zone_14,
                }

    pitch_pref_data.append(zone_eff_h)

In [104]:
pitch_trend_data = []
for idp in players.id:
    
    p = data[data["pitcher_id"] == idp]

    zone_1 = p[p["pitch_location_zone"] == 1]["y_target"].mean()
    zone_2 = p[p["pitch_location_zone"] == 2]["y_target"].mean()
    zone_3 = p[p["pitch_location_zone"] == 3]["y_target"].mean()
    zone_4 = p[p["pitch_location_zone"] == 4]["y_target"].mean()
    zone_5 = p[p["pitch_location_zone"] == 5]["y_target"].mean()
    zone_6 = p[p["pitch_location_zone"] == 6]["y_target"].mean()
    zone_7 = p[p["pitch_location_zone"] == 7]["y_target"].mean()
    zone_8 = p[p["pitch_location_zone"] == 8]["y_target"].mean()
    zone_9 = p[p["pitch_location_zone"] == 9]["y_target"].mean()
    zone_10 = p[p["pitch_location_zone"] == 10]["y_target"].mean()
    zone_11 = p[p["pitch_location_zone"] == 11]["y_target"].mean()
    zone_12 = p[p["pitch_location_zone"] == 12]["y_target"].mean()
    zone_13 = p[p["pitch_location_zone"] == 13]["y_target"].mean()
    zone_14 = p[p["pitch_location_zone"] == 14]["y_target"].mean()
    

    zone_eff_p = {'id': idp,
                'zone_1': zone_1,
                'zone_2': zone_2,
                'zone_3': zone_3,
                'zone_4': zone_4,
                'zone_5': zone_5,
                'zone_6': zone_6,
                'zone_7': zone_7,
                'zone_8': zone_8,
                'zone_9': zone_9,
                'zone_10': zone_10,
                'zone_11': zone_11,
                'zone_12': zone_12,
                'zone_13': zone_13,
                'zone_14': zone_14,
                }

    pitch_trend_data.append(zone_eff_p)

In [105]:
# Merging data and hitters
hitters = players[~players.id.duplicated(keep="first")]
hitter_pitch_pref = pd.DataFrame(pitch_pref_data)
hitter_pitch_pref = hitter_pitch_pref.fillna(0)
hitters = hitters.merge(hitter_pitch_pref, how='left', on="id")

hitters = hitters.add_prefix("hitter_")
data = data.merge(hitters, how="left", on="hitter_id")

In [106]:
data.head()

,id,game_id,inning,side,hitter_id,hitter_hand,pitcher_id,pitcher_hand,description,temp_f,weather_condition,humidity,wind_speed_mph,at_bat_end_time,pitch_location_zone,pitch_type_code,pitch_type_des,pitch_speed_mph,pitch_count_at_bat,pitcher_pitch_count_at_bat_start,outs_at_start,output_code,play_outcome,mc_target,y_target,status,coverage,game_number,day_night,scheduled,home_team,away_team,attendance,duration,double_header,entry_mode,reference,venue,home,away,broadcast,rescheduled,venue_id,hitter_player_name,hitter_team_id,hitter_team_name,hitter_position,hitter_primary_position,hitter_zone_1,hitter_zone_2,hitter_zone_3,hitter_zone_4,hitter_zone_5,hitter_zone_6,hitter_zone_7,hitter_zone_8,hitter_zone_9,hitter_zone_10,hitter_zone_11,hitter_zone_12,hitter_zone_13,hitter_zone_14
0,67cfd85d-029a-4e5d-9ad3-cf7c1f29c303,e60d1770-2789-4d00-b94a-5033a7fa89a9,1,T,084d2514-9ffb-414e-ae16-3bc690aaad51,R,e28344cd-b08e-43ec-adcb-f6693203cf82,R,Mookie Betts flies out to deep left field to C...,51.0,"Light Rain, Mist",96.0,4.0,2023-05-08T23:43:05+00:00,2.0,FF,Four-Seam Fastball,96.5,4.0,0.0,0.0,oFO,IPO,0,0,closed,full,1,N,2023-05-08T23:40:00+00:00,dcfd5266-00ce-442c-bc09-264cd20cf455,ef64da7f-cfaf-4300-87b0-9313386b977c,22847.0,2:18,False,STOMP,718257,"{'name': 'American Family Field', 'market': 'M...","{'name': 'Brewers', 'market': 'Milwaukee', 'ab...","{'name': 'Dodgers', 'market': 'Los Angeles', '...",{'network': 'MLB Network'},NaN,3d13c8a7-283f-482b-ade1-441e25b6465d,"Betts, Markus",ef64da7f-cfaf-4300-87b0-9313386b977c,Dodgers,OF,RF,0.296296,0.272727,0.322581,0.500000,0.267606,0.259259,0.464286,0.428571,0.300000,0.0,0.775000,0.555556,0.500000,0.370370
1,1ec4235d-7ee4-47f7-a262-40ef0a76acd2,e60d1770-2789-4d00-b94a-5033a7fa89a9,1,T,ea1a2111-44cc-4996-babb-9439465e6760,L,e28344cd-b08e-43ec-adcb-f6693203cf82,R,Freddie Freeman lines out to right field to Ty...,51.0,"Light Rain, Mist",96.0,4.0,2023-05-08T23:45:01+00:00,13.0,CU,Curveball,76.3,5.0,4.0,1.0,oLO,IPO,0,0,closed,full,1,N,2023-05-08T23:40:00+00:00,dcfd5266-00ce-442c-bc09-264cd20cf455,ef64da7f-cfaf-4300-87b0-9313386b977c,22847.0,2:18,False,STOMP,718257,"{'name': 'American Family Field', 'market': 'M...","{'name': 'Brewers', 'market': 'Milwaukee', 'ab...","{'name': 'Dodgers', 'market': 'Los Angeles', '...",{'network': 'MLB Network'},NaN,3d13c8a7-283f-482b-ade1-441e25b6465d,"Freeman, Frederick",ef64da7f-cfaf-4300-87b0-9313386b977c,Dodgers,IF,1B,0.545455,0.264706,0.285714,0.437500,0.518519,0.425000,0.192308,0.343284,0.461538,0.0,0.500000,0.500000,0.424658,0.430769
2,d5d817fa-a490-45b3-9884-0e6a70d97e3e,e60d1770-2789-4d00-b94a-5033a7fa89a9,1,T,8e42fd09-b9d0-4566-b960-e107f580de46,L,e28344cd-b08e-43ec-adcb-f6693203cf82,R,"Jason Heyward grounds out to second base, Bric...",51.0,"Light Rain, Mist",96.0,4.0,2023-05-08T23:45:36+00:00,9.0,FF,Four-Seam Fastball,94.8,1.0,9.0,2.0,oGO,IPO,0,0,closed,full,1,N,2023-05-08T23:40:00+00:00,dcfd5266-00ce-442c-bc09-264cd20cf455,ef64da7f-cfaf-4300-87b0-9313386b977c,22847.0,2:18,False,STOMP,718257,"{'name': 'American Family Field', 'market': 'M...","{'name': 'Brewers', 'market': 'Milwaukee', 'ab...","{'name': 'Dodgers', 'market': 'Los Angeles', '...",{'network': 'MLB Network'},NaN,3d13c8a7-283f-482b-ade1-441e25b6465d,"Heyward, Jason",ef64da7f-cfaf-4300-87b0-9313386b977c,Dodgers,OF,RF,0.200000,0.375000,0.333333,0.157895,0.185185,0.312500,0.238095,0.272727,0.368421,0.0,0.733333,0.500000,0.277778,0.484848
3,c4cfeef1-8fa9-4ae4-aa35-c56dabe5bde4,e60d1770-2789-4d00-b94a-5033a7fa89a9,1,B,42cb5171-ffa3-4600-9c41-dbc3805206ea,L,c144f063-3940-472c-b607-9e5359844431,R,Christian Yelich flies out to deep right field...,51.0,"Light Rain, Mist",96.0,4.0,2023-05-08T23:47:55+00:00,5.0,FF,Four-Seam Fastball,91.8,1.0,0.0,0.0,oFO,IPO,0,0,closed,full,1,N,2023-05-08T23:40:00+00:00,dcfd5266-00ce-442c-bc09-264cd20cf455,ef64da7f-cfaf-4300-87b0-9313386b977c,22847.0,2:18,False,STOMP,718257,"{'name': 'American Family Field', 'market': 'M...","{'name': 'Brewers', 'market': 'Milwaukee', 'a

In [107]:
# Merging data and pitchers
pitchers = players[~players.id.duplicated(keep="first")]

pitcher_pitch_trend = pd.DataFrame(pitch_trend_data)
pitcher_pitch_trend  = pitcher_pitch_trend.fillna(0)
pitchers = pitchers.merge(pitcher_pitch_trend, how='left', on="id")
pitchers = pitchers.add_prefix("pitcher_")
data = data.merge(pitchers, how="left", on="pitcher_id")
data.shape

(142307, 81)

In [108]:
# Merging data and home team
home_team = teams.add_prefix("home_team_")
data = data.rename(columns={"home_team": "home_team_id"})
data = data.merge(home_team, how="left", on="home_team_id")
data.shape

(142307, 84)

In [109]:
# Merging data and away team
away_team = teams.add_prefix("away_team_")
data = data.rename(columns={"away_team": "away_team_id"})
data = data.merge(away_team, how="left", on="away_team_id")
data.shape

(142307, 87)

In [110]:
# Merging data and stadium
venue = stadiums.add_prefix("stadium_")
data = data.rename(columns={"venue_id": "stadium_id"})
data = data.merge(venue, how="left", on="stadium_id")
data.shape

(142307, 101)

In [111]:
# Merging data and away adress
away_stadium = stadiums
away_stadium["abbr"] = teams.abbr
away_stadium = away_stadium.add_prefix("away_stadium_")
away_stadium = away_stadium.rename(columns={"away_stadium_abbr": "away_team_abbr"})
data = data.merge(away_stadium, how="left", on="away_team_abbr")
data.shape

(142307, 116)

In [112]:
data_all = data
data_all.to_csv('../raw_data/all_data_merged.csv', index=True)

In [113]:
data.head(3)

,id,game_id,inning,side,hitter_id,hitter_hand,pitcher_id,pitcher_hand,description,temp_f,weather_condition,humidity,wind_speed_mph,at_bat_end_time,pitch_location_zone,pitch_type_code,pitch_type_des,pitch_speed_mph,pitch_count_at_bat,pitcher_pitch_count_at_bat_start,outs_at_start,output_code,play_outcome,mc_target,y_target,status,coverage,game_number,day_night,scheduled,home_team_id,away_team_id,attendance,duration,double_header,entry_mode,reference,venue,home,away,broadcast,rescheduled,stadium_id,hitter_player_name,hitter_team_id,hitter_team_name,hitter_position,hitter_primary_position,hitter_zone_1,hitter_zone_2,hitter_zone_3,hitter_zone_4,hitter_zone_5,hitter_zone_6,hitter_zone_7,hitter_zone_8,hitter_zone_9,hitter_zone_10,hitter_zone_11,hitter_zone_12,hitter_zone_13,hitter_zone_14,pitcher_player_name,pitcher_team_id,pitcher_team_name,pitcher_position,pitcher_primary_position,pitcher_zone_1,pitcher_zone_2,pitcher_zone_3,pitcher_zone_4,pitcher_zone_5,pitcher_zone_6,pitcher_zone_7,pitcher_zone_8,pitcher_zone_9,pitcher_zone_10,pitcher_zone_11,pitcher_zone_12,pitcher_zone_13,pitcher_zone_14,home_team_name,home_team_market,home_team_abbr,away_team_name,away_team_market,away_team_abbr,stadium_name,stadium_market,stadium_capacity,stadium_surface,stadium_address,stadium_city,stadium_state,stadium_zip,stadium_country,stadium_field_orientation,stadium_stadium_type,stadium_time_zone,stadium_lat,stadium_lon,away_stadium_id,away_stadium_name,away_stadium_market,away_stadium_capacity,away_stadium_surface,away_stadium_address,away_stadium_city,away_stadium_state,away_stadium_zip,away_stadium_country,away_stadium_field_orientation,away_stadium_stadium_type,away_stadium_time_zone,away_stadium_lat,away_stadium_lon
0,67cfd85d-029a-4e5d-9ad3-cf7c1f29c303,e60d1770-2789-4d00-b94a-5033a7fa89a9,1,T,084d2514-9ffb-414e-ae16-3bc690aaad51,R,e28344cd-b08e-43ec-adcb-f6693203cf82,R,Mookie Betts flies out to deep left field to C...,51.0,"Light Rain, Mist",96.0,4.0,2023-05-08T23:43:05+00:00,2.0,FF,Four-Seam Fastball,96.5,4.0,0.0,0.0,oFO,IPO,0,0,closed,full,1,N,2023-05-08T23:40:00+00:00,dcfd5266-00ce-442c-bc09-264cd20cf455,ef64da7f-cfaf-4300-87b0-9313386b977c,22847.0,2:18,False,STOMP,718257,"{'name': 'American Family Field', 'market': 'M...","{'name': 'Brewers', 'market': 'Milwaukee', 'ab...","{'name': 'Dodgers', 'market': 'Los Angeles', '...",{'network': 'MLB Network'},NaN,3d13c8a7-283f-482b-ade1-441e25b6465d,"Betts, Markus",ef64da7f-cfaf-4300-87b0-9313386b977c,Dodgers,OF,RF,0.296296,0.272727,0.322581,0.500000,0.267606,0.259259,0.464286,0.428571,0.300000,0.0,0.775000,0.555556,0.500000,0.370370,"Peralta, Freddy",dcfd5266-00ce-442c-bc09-264cd20cf455,Brewers,P,SP,0.176471,0.2,0.428571,0.258065,0.307692,0.25,0.25,0.314815,0.294118,0.0,0.47541,0.25,0.318182,0.258621,Brewers,Milwaukee,MIL,Dodgers,Los Angeles,LAD,American Family Field,Milwaukee,41700.0,grass,One Brewers Way,Milwaukee,WI,53214,USA,SE,retractable,US/Central,43.028739,-87.971303,66a19c3d-24fe-477d-bee7-c6ef1b98352f,Dodger Stadium,Los Angeles,56000.0,grass,1000 Vin Scully Avenue,Los Angeles,CA,90012,USA,NE,outdoor,US/Pacific,34.074541,-118.240888
1,1ec4235d-7ee4-47f7-a262-40ef0a76acd2,e60d1770-2789-4d00-b94a-5033a7fa89a9,1,T,ea1a2111-44cc-4996-babb-9439465e6760,L,e28344cd-b08e-43ec-adcb-f6693203cf82,R,Freddie Freeman lines out to right field to Ty...,51.0,"Light Rain, Mist",96.0,4.0,2023-05-08T23:45:01+00:00,13.0,CU,Curveball,76.3,5.0,4.0,1.0,oLO,IPO,0,0,closed,full,1,N,2023-05-08T23:40:00+00:00,dcfd5266-00ce-442c-bc09-264cd20cf455,ef64da7f-cfaf-4300-87b0-9313386b977c,22847.0,2:18,False,STOMP,718257,"{'name': 'American Family Field', 'market': 'M...","{'name': 'Brewers', 'market': 'Milwaukee', 'ab...","{'name': 'Dodgers', 'market': 'Los Angeles', '...",{'network': 'MLB Network'},NaN,3d13c8a7-283f-482b-ade1-441e25b6465d,"Freeman, Frederick",ef64da7f-cfaf-4300-87b0-9313386b977c,Dodgers,IF,1B,0.545455,0.264706,0.285714,0.437500,0.518519,0.425000,0.192308,0.343284,0.461538,0.0,0.500000,0.500000,0.424658,0.

In [114]:
data.describe()

,inning,temp_f,humidity,wind_speed_mph,pitch_location_zone,pitch_speed_mph,pitch_count_at_bat,pitcher_pitch_count_at_bat_start,outs_at_start,mc_target,y_target,game_number,attendance,reference,hitter_zone_1,hitter_zone_2,hitter_zone_3,hitter_zone_4,hitter_zone_5,hitter_zone_6,hitter_zone_7,hitter_zone_8,hitter_zone_9,hitter_zone_10,hitter_zone_11,hitter_zone_12,hitter_zone_13,hitter_zone_14,pitcher_zone_1,pitcher_zone_2,pitcher_zone_3,pitcher_zone_4,pitcher_zone_5,pitcher_zone_6,pitcher_zone_7,pitcher_zone_8,pitcher_zone_9,pitcher_zone_10,pitcher_zone_11,pitcher_zone_12,pitcher_zone_13,pitcher_zone_14,stadium_capacity,stadium_lat,stadium_lon,away_stadium_capacity,away_stadium_lat,away_stadium_lon
count,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,141860.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.0,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.0,142307.000000,142307.000000,142307.000000,142307.000000,141918.000000,141918.000000,141918.000000,142232.000000,142232.000000,142232.000000
mean,4.950733,73.288419,54.083784,11.195423,7.856016,89.056774,3.896288,29.474291,0.966249,0.464938,0.317061,1.014764,29309.146870,717835.681878,0.264690,0.271152,0.261591,0.292933,0.329638,0.289764,0.272492,0.305614,0.253144,0.0,0.476343,0.401911,0.337046,0.348882,0.263043,0.268810,0.258576,0.291229,0.326383,0.285395,0.265639,0.305686,0.248992,0.0,0.468260,0.373844,0.334105,0.338397,42386.568969,38.125192,-92.608178,42388.579989,38.224083,-92.399080
std,2.579747,12.714632,19.634731,221.839228,4.076619,6.106812,1.892109,27.078681,0.817366,0.861797,0.465333,0.120607,11309.926713,546.487728,0.146445,0.123189,0.140488,0.107611,0.093800,0.108369,0.131476,0.104908,0.114648,0.0,0.159291,0.160720,0.123411,0.122465,0.168488,0.149591,0.172543,0.122082,0.110788,0.131162,0.147037,0.121093,0.136733,0.0,0.182810,0.193644,0.157708,0.140193,5477.181213,5.018912,16.401666,5487.732362,5.033276,16.416360
min,1.000000,34.000000,4.000000,1.000000,1.000000,33.700000,1.000000,-6.000000,0.000000,0.000000,0.000000,1.000000,0.000000,716887.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,25025.000000,25.778057,-122.390621,25025.000000,25.778057,-122.390621
25%,3.000000,65.000000,40.000000,4.000000,5.000000,84.600000,2.000000,7.000000,0.000000,0.000000,0.000000,1.000000,20286.000000,717364.000000,0.176471,0.200000,0.181818,0.230769,0.272727,0.235294,0.200000,0.250000,0.181818,0.0,0.375000,0.300000,0.260870,0.269231,0.166667,0.185714,0.142857,0.222222,0.272727,0.210526,0.175000,0.242424,0.176471,0.0,0.363636,0.258065,0.230769,0.258065,40000.000000,33.890672,-104.993349,40000.000000,33.890672,-104.993349
50%,5.000000,74.000000,56.000000,8.000000,8.000000,89.800000,4.000000,20.000000,1.000000,0.000000,0.000000,1.000000,30578.000000,717834.000000,0.250000,0.268293,0.263158,0.293103,0.333333,0.291667,0.266667,0.307692,0.257143,0.0,0.473684,0.414634,0.333333,0.338710,0.250000,0.263158,0.250000,0.285714,0.326531,0.277778,0.259259,0.300000,0.240741,0.0,0.465517,0.360000,0.322581,0.320000,41376.000000,39.097736,-87.656054,41700.000000,39.283787,-87.634833
75%,7.000000,82.000000,69.000000,12.000000,12.000000,93.900000,5.000000,50.000000,2.000000,1.000000,1.000000,1.000000,38617.000000,718308.000000,0.333333,0.333333,0.333333,0.352941,0.386364,0.355556,0.352941,0.370370,0.318182,0.0,0.586207,0.500000,0.410256,0.425926,0.344828,0.340909,0.333333,0.352941,0.388889,0.350000,0.344828,0.368421,0.312500,0.0,0.571429,0.500000,

In [115]:
data.duplicated().sum()

0

In [120]:
columns_to_remove_fp = list(('description', 'play_outcome', 'mc_target',
                            'status', 'coverage', 'game_number', 
                            'duration', 'double_header', 'entry_mode', 'reference', 
                            'venue', 'home', 'away', 'broadcast', 'rescheduled','hitter_team_id', 'hitter_team_name','pitcher_position',
                            'pitcher_team_id', 'pitcher_team_name', 'home_team_name', 'home_team_market', 'home_team_abbr',
                            'away_team_name', 'away_team_market', 'away_team_abbr', 'stadium_name', 'stadium_market', 'stadium_surface', 'stadium_address',
                            'stadium_city', 'stadium_state', 'stadium_zip', 'stadium_country', 'stadium_field_orientation', 'stadium_time_zone', 'away_stadium_id',
                            'away_stadium_name', 'away_stadium_market', 'away_stadium_surface', 'away_stadium_address', 'away_stadium_city', 'away_stadium_state', 'away_stadium_zip',
                            'away_stadium_country', 'away_stadium_field_orientation', 'away_stadium_stadium_type', 'away_stadium_time_zone', 'pitch_type_des'))

In [121]:
data = data.drop(columns=columns_to_remove_fp)

In [122]:
#Cleaning up data points
data['outs_at_start'] = data['outs_at_start'].apply(lambda x: 2 if x == 3 else x)
data['pitcher_pitch_count_at_bat_start'] = data['pitcher_pitch_count_at_bat_start'].apply(lambda x: 0 if x < 0 else x)
data['wind_speed_mph'] = data['wind_speed_mph'].apply(lambda x: 50 if x > 50 else x)

In [123]:
data.describe()

,inning,temp_f,humidity,wind_speed_mph,pitch_location_zone,pitch_speed_mph,pitch_count_at_bat,pitcher_pitch_count_at_bat_start,outs_at_start,y_target,attendance,hitter_zone_1,hitter_zone_2,hitter_zone_3,hitter_zone_4,hitter_zone_5,hitter_zone_6,hitter_zone_7,hitter_zone_8,hitter_zone_9,hitter_zone_10,hitter_zone_11,hitter_zone_12,hitter_zone_13,hitter_zone_14,pitcher_zone_1,pitcher_zone_2,pitcher_zone_3,pitcher_zone_4,pitcher_zone_5,pitcher_zone_6,pitcher_zone_7,pitcher_zone_8,pitcher_zone_9,pitcher_zone_10,pitcher_zone_11,pitcher_zone_12,pitcher_zone_13,pitcher_zone_14,stadium_capacity,stadium_lat,stadium_lon,away_stadium_capacity,away_stadium_lat,away_stadium_lon
count,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,141860.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.0,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.000000,142307.0,142307.000000,142307.000000,142307.000000,142307.000000,141918.000000,141918.000000,141918.000000,142232.000000,142232.000000,142232.000000
mean,4.950733,73.288419,54.083784,8.972419,7.856016,89.056774,3.896288,29.476069,0.966221,0.317061,29309.146870,0.264690,0.271152,0.261591,0.292933,0.329638,0.289764,0.272492,0.305614,0.253144,0.0,0.476343,0.401911,0.337046,0.348882,0.263043,0.268810,0.258576,0.291229,0.326383,0.285395,0.265639,0.305686,0.248992,0.0,0.468260,0.373844,0.334105,0.338397,42386.568969,38.125192,-92.608178,42388.579989,38.224083,-92.399080
std,2.579747,12.714632,19.634731,5.069735,4.076619,6.106812,1.892109,27.076677,0.817314,0.465333,11309.926713,0.146445,0.123189,0.140488,0.107611,0.093800,0.108369,0.131476,0.104908,0.114648,0.0,0.159291,0.160720,0.123411,0.122465,0.168488,0.149591,0.172543,0.122082,0.110788,0.131162,0.147037,0.121093,0.136733,0.0,0.182810,0.193644,0.157708,0.140193,5477.181213,5.018912,16.401666,5487.732362,5.033276,16.416360
min,1.000000,34.000000,4.000000,1.000000,1.000000,33.700000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,25025.000000,25.778057,-122.390621,25025.000000,25.778057,-122.390621
25%,3.000000,65.000000,40.000000,4.000000,5.000000,84.600000,2.000000,7.000000,0.000000,0.000000,20286.000000,0.176471,0.200000,0.181818,0.230769,0.272727,0.235294,0.200000,0.250000,0.181818,0.0,0.375000,0.300000,0.260870,0.269231,0.166667,0.185714,0.142857,0.222222,0.272727,0.210526,0.175000,0.242424,0.176471,0.0,0.363636,0.258065,0.230769,0.258065,40000.000000,33.890672,-104.993349,40000.000000,33.890672,-104.993349
50%,5.000000,74.000000,56.000000,8.000000,8.000000,89.800000,4.000000,20.000000,1.000000,0.000000,30578.000000,0.250000,0.268293,0.263158,0.293103,0.333333,0.291667,0.266667,0.307692,0.257143,0.0,0.473684,0.414634,0.333333,0.338710,0.250000,0.263158,0.250000,0.285714,0.326531,0.277778,0.259259,0.300000,0.240741,0.0,0.465517,0.360000,0.322581,0.320000,41376.000000,39.097736,-87.656054,41700.000000,39.283787,-87.634833
75%,7.000000,82.000000,69.000000,12.000000,12.000000,93.900000,5.000000,50.000000,2.000000,1.000000,38617.000000,0.333333,0.333333,0.333333,0.352941,0.386364,0.355556,0.352941,0.370370,0.318182,0.0,0.586207,0.500000,0.410256,0.425926,0.344828,0.340909,0.333333,0.352941,0.388889,0.350000,0.344828,0.368421,0.312500,0.0,0.571429,0.500000,0.411765,0.407407,45971.000000,41.830066,-80.006409,45971.000000,41.830066,-80.006409
max,14.000000,117.000000,100.000000,50.000000,14.000000,104.800000,16.000000,117.000000,2.000000,1.000000,55565.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000

In [124]:
data.head(2)

,id,game_id,inning,side,hitter_id,hitter_hand,pitcher_id,pitcher_hand,temp_f,weather_condition,humidity,wind_speed_mph,at_bat_end_time,pitch_location_zone,pitch_type_code,pitch_speed_mph,pitch_count_at_bat,pitcher_pitch_count_at_bat_start,outs_at_start,output_code,y_target,day_night,scheduled,home_team_id,away_team_id,attendance,stadium_id,hitter_player_name,hitter_position,hitter_primary_position,hitter_zone_1,hitter_zone_2,hitter_zone_3,hitter_zone_4,hitter_zone_5,hitter_zone_6,hitter_zone_7,hitter_zone_8,hitter_zone_9,hitter_zone_10,hitter_zone_11,hitter_zone_12,hitter_zone_13,hitter_zone_14,pitcher_player_name,pitcher_primary_position,pitcher_zone_1,pitcher_zone_2,pitcher_zone_3,pitcher_zone_4,pitcher_zone_5,pitcher_zone_6,pitcher_zone_7,pitcher_zone_8,pitcher_zone_9,pitcher_zone_10,pitcher_zone_11,pitcher_zone_12,pitcher_zone_13,pitcher_zone_14,stadium_capacity,stadium_stadium_type,stadium_lat,stadium_lon,away_stadium_capacity,away_stadium_lat,away_stadium_lon
0,67cfd85d-029a-4e5d-9ad3-cf7c1f29c303,e60d1770-2789-4d00-b94a-5033a7fa89a9,1,T,084d2514-9ffb-414e-ae16-3bc690aaad51,R,e28344cd-b08e-43ec-adcb-f6693203cf82,R,51.0,"Light Rain, Mist",96.0,4.0,2023-05-08T23:43:05+00:00,2.0,FF,96.5,4.0,0.0,0.0,oFO,0,N,2023-05-08T23:40:00+00:00,dcfd5266-00ce-442c-bc09-264cd20cf455,ef64da7f-cfaf-4300-87b0-9313386b977c,22847.0,3d13c8a7-283f-482b-ade1-441e25b6465d,"Betts, Markus",OF,RF,0.296296,0.272727,0.322581,0.5000,0.267606,0.259259,0.464286,0.428571,0.300000,0.0,0.775,0.555556,0.500000,0.370370,"Peralta, Freddy",SP,0.176471,0.2,0.428571,0.258065,0.307692,0.25,0.25,0.314815,0.294118,0.0,0.47541,0.25,0.318182,0.258621,41700.0,retractable,43.028739,-87.971303,56000.0,34.074541,-118.240888
1,1ec4235d-7ee4-47f7-a262-40ef0a76acd2,e60d1770-2789-4d00-b94a-5033a7fa89a9,1,T,ea1a2111-44cc-4996-babb-9439465e6760,L,e28344cd-b08e-43ec-adcb-f6693203cf82,R,51.0,"Light Rain, Mist",96.0,4.0,2023-05-08T23:45:01+00:00,13.0,CU,76.3,5.0,4.0,1.0,oLO,0,N,2023-05-08T23:40:00+00:00,dcfd5266-00ce-442c-bc09-264cd20cf455,ef64da7f-cfaf-4300-87b0-9313386b977c,22847.0,3d13c8a7-283f-482b-ade1-441e25b6465d,"Freeman, Frederick",IF,1B,0.545455,0.264706,0.285714,0.4375,0.518519,0.425000,0.192308,0.343284,0.461538,0.0,0.500,0.500000,0.424658,0.430769,"Peralta, Freddy",SP,0.176471,0.2,0.428571,0.258065,0.307692,0.25,0.25,0.314815,0.294118,0.0,0.47541,0.25,0.318182,0.258621,41700.0,retractable,43.028739,-87.971303,56000.0,34.074541,-118.240888


In [125]:
#Coverting columns to the correct dtype
data["scheduled"] = pd.to_datetime(data["scheduled"])
data["at_bat_end_time"] = pd.to_datetime(data["at_bat_end_time"])

In [126]:
data.head(2)

,id,game_id,inning,side,hitter_id,hitter_hand,pitcher_id,pitcher_hand,temp_f,weather_condition,humidity,wind_speed_mph,at_bat_end_time,pitch_location_zone,pitch_type_code,pitch_speed_mph,pitch_count_at_bat,pitcher_pitch_count_at_bat_start,outs_at_start,output_code,y_target,day_night,scheduled,home_team_id,away_team_id,attendance,stadium_id,hitter_player_name,hitter_position,hitter_primary_position,hitter_zone_1,hitter_zone_2,hitter_zone_3,hitter_zone_4,hitter_zone_5,hitter_zone_6,hitter_zone_7,hitter_zone_8,hitter_zone_9,hitter_zone_10,hitter_zone_11,hitter_zone_12,hitter_zone_13,hitter_zone_14,pitcher_player_name,pitcher_primary_position,pitcher_zone_1,pitcher_zone_2,pitcher_zone_3,pitcher_zone_4,pitcher_zone_5,pitcher_zone_6,pitcher_zone_7,pitcher_zone_8,pitcher_zone_9,pitcher_zone_10,pitcher_zone_11,pitcher_zone_12,pitcher_zone_13,pitcher_zone_14,stadium_capacity,stadium_stadium_type,stadium_lat,stadium_lon,away_stadium_capacity,away_stadium_lat,away_stadium_lon
0,67cfd85d-029a-4e5d-9ad3-cf7c1f29c303,e60d1770-2789-4d00-b94a-5033a7fa89a9,1,T,084d2514-9ffb-414e-ae16-3bc690aaad51,R,e28344cd-b08e-43ec-adcb-f6693203cf82,R,51.0,"Light Rain, Mist",96.0,4.0,2023-05-08 23:43:05+00:00,2.0,FF,96.5,4.0,0.0,0.0,oFO,0,N,2023-05-08 23:40:00+00:00,dcfd5266-00ce-442c-bc09-264cd20cf455,ef64da7f-cfaf-4300-87b0-9313386b977c,22847.0,3d13c8a7-283f-482b-ade1-441e25b6465d,"Betts, Markus",OF,RF,0.296296,0.272727,0.322581,0.5000,0.267606,0.259259,0.464286,0.428571,0.300000,0.0,0.775,0.555556,0.500000,0.370370,"Peralta, Freddy",SP,0.176471,0.2,0.428571,0.258065,0.307692,0.25,0.25,0.314815,0.294118,0.0,0.47541,0.25,0.318182,0.258621,41700.0,retractable,43.028739,-87.971303,56000.0,34.074541,-118.240888
1,1ec4235d-7ee4-47f7-a262-40ef0a76acd2,e60d1770-2789-4d00-b94a-5033a7fa89a9,1,T,ea1a2111-44cc-4996-babb-9439465e6760,L,e28344cd-b08e-43ec-adcb-f6693203cf82,R,51.0,"Light Rain, Mist",96.0,4.0,2023-05-08 23:45:01+00:00,13.0,CU,76.3,5.0,4.0,1.0,oLO,0,N,2023-05-08 23:40:00+00:00,dcfd5266-00ce-442c-bc09-264cd20cf455,ef64da7f-cfaf-4300-87b0-9313386b977c,22847.0,3d13c8a7-283f-482b-ade1-441e25b6465d,"Freeman, Frederick",IF,1B,0.545455,0.264706,0.285714,0.4375,0.518519,0.425000,0.192308,0.343284,0.461538,0.0,0.500,0.500000,0.424658,0.430769,"Peralta, Freddy",SP,0.176471,0.2,0.428571,0.258065,0.307692,0.25,0.25,0.314815,0.294118,0.0,0.47541,0.25,0.318182,0.258621,41700.0,retractable,43.028739,-87.971303,56000.0,34.074541,-118.240888


In [127]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142307 entries, 0 to 142306
Data columns (total 67 columns):
 #   Column                            Non-Null Count   Dtype              
---  ------                            --------------   -----              
 0   id                                142307 non-null  object             
 1   game_id                           142307 non-null  object             
 2   inning                            142307 non-null  int64              
 3   side                              142307 non-null  object             
 4   hitter_id                         142307 non-null  object             
 5   hitter_hand                       142307 non-null  object             
 6   pitcher_id                        142307 non-null  object             
 7   pitcher_hand                      142307 non-null  object             
 8   temp_f                            142307 non-null  float64            
 9   weather_condition                 142307 non-nul

In [128]:
data = data.sort_values(["at_bat_end_time", "inning"], ignore_index=True, ascending=True)

In [129]:
data.head(2)

,id,game_id,inning,side,hitter_id,hitter_hand,pitcher_id,pitcher_hand,temp_f,weather_condition,humidity,wind_speed_mph,at_bat_end_time,pitch_location_zone,pitch_type_code,pitch_speed_mph,pitch_count_at_bat,pitcher_pitch_count_at_bat_start,outs_at_start,output_code,y_target,day_night,scheduled,home_team_id,away_team_id,attendance,stadium_id,hitter_player_name,hitter_position,hitter_primary_position,hitter_zone_1,hitter_zone_2,hitter_zone_3,hitter_zone_4,hitter_zone_5,hitter_zone_6,hitter_zone_7,hitter_zone_8,hitter_zone_9,hitter_zone_10,hitter_zone_11,hitter_zone_12,hitter_zone_13,hitter_zone_14,pitcher_player_name,pitcher_primary_position,pitcher_zone_1,pitcher_zone_2,pitcher_zone_3,pitcher_zone_4,pitcher_zone_5,pitcher_zone_6,pitcher_zone_7,pitcher_zone_8,pitcher_zone_9,pitcher_zone_10,pitcher_zone_11,pitcher_zone_12,pitcher_zone_13,pitcher_zone_14,stadium_capacity,stadium_stadium_type,stadium_lat,stadium_lon,away_stadium_capacity,away_stadium_lat,away_stadium_lon
0,d43f7325-ca19-49ab-94f6-b92055c26dd5,f3f0ae8e-cb65-4e96-9530-c6f868738f09,1,T,041632a9-afb2-4ec3-b1de-9b0bbe33ab64,L,eebc991a-23ea-4f1c-ba3b-37ff21ee1603,R,38.0,Sunny,32.0,9.0,2023-03-30 17:11:07+00:00,11.0,FF,94.4,4.0,0.0,0.0,bB,1,D,2023-03-30 17:05:00+00:00,a09ec676-f887-43dc-bbb3-cf4bbaee9a18,a7723160-10b7-4277-a309-d8dd95a8ae65,46172.0,706e9828-6687-4ac8-a409-3fb972e8bae9,"Wade Jr., LaMonte",IF,1B,0.312500,0.227273,0.454545,0.268293,0.37500,0.205882,0.342105,0.333333,0.272727,0.0,0.638298,0.416667,0.452381,0.40625,"Cole, Gerrit",SP,0.15,0.173913,0.25,0.135135,0.28,0.333333,0.15625,0.333333,0.226415,0.0,0.446154,0.365854,0.190476,0.296296,47309.0,outdoor,40.828819,-73.926569,41915.0,37.77842,-122.390621
1,f7f6d34f-5e1e-4ca9-896a-1f026054cbf5,f3f0ae8e-cb65-4e96-9530-c6f868738f09,1,T,e5bdeb0e-38fc-4d30-8127-43d0d5b2864d,L,eebc991a-23ea-4f1c-ba3b-37ff21ee1603,R,38.0,Sunny,32.0,9.0,2023-03-30 17:13:06+00:00,8.0,SL,86.8,5.0,4.0,0.0,kKL,0,D,2023-03-30 17:05:00+00:00,a09ec676-f887-43dc-bbb3-cf4bbaee9a18,a7723160-10b7-4277-a309-d8dd95a8ae65,46172.0,706e9828-6687-4ac8-a409-3fb972e8bae9,"Conforto, Michael",OF,RF,0.222222,0.269231,0.294118,0.204545,0.27907,0.272727,0.240000,0.341463,0.260870,0.0,0.577778,0.391304,0.347826,0.50000,"Cole, Gerrit",SP,0.15,0.173913,0.25,0.135135,0.28,0.333333,0.15625,0.333333,0.226415,0.0,0.446154,0.365854,0.190476,0.296296,47309.0,outdoor,40.828819,-73.926569,41915.0,37.77842,-122.390621


In [130]:
secondary_drop = list(('scheduled', 'away_stadium_capacity', 'pitch_type_code', 
                            'output_code'))

In [132]:
data = data.drop(columns=secondary_drop)

KeyError: "['scheduled', 'away_stadium_capacity', 'pitch_type_code', 'output_code'] not found in axis"

In [133]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142307 entries, 0 to 142306
Data columns (total 63 columns):
 #   Column                            Non-Null Count   Dtype              
---  ------                            --------------   -----              
 0   id                                142307 non-null  object             
 1   game_id                           142307 non-null  object             
 2   inning                            142307 non-null  int64              
 3   side                              142307 non-null  object             
 4   hitter_id                         142307 non-null  object             
 5   hitter_hand                       142307 non-null  object             
 6   pitcher_id                        142307 non-null  object             
 7   pitcher_hand                      142307 non-null  object             
 8   temp_f                            142307 non-null  float64            
 9   weather_condition                 142307 non-nul

In [134]:
data.to_csv('../raw_data/final_raw_data.csv', index=True)